## Collecting Data from Reddit.com
#### Using PSAW and PRAW in Python

In [1]:
!pip3 install praw
#pip install praw #Jupyter
#conda install -c conda-forge praw #Anaconda
import praw

     ------------------------------------ 188.2/188.2 kB 669.0 kB/s eta 0:00:00


In [69]:
!pip3 install psaw

# Get Credentials:
https://old.reddit.com/prefs/apps/

In [15]:
# Log In to App: 
reddit = praw.Reddit(client_id='9MKSo0tYTRaUWPrcNq8tpA', 
                     client_secret='5twxWoEj6Y4fvik1SbqA7Sac105PNg', 
                     user_agent='price App')

In [16]:
print(reddit.read_only)

True


### Authorized Reddit Instances

In [17]:
reddit = praw.Reddit(
    client_id="9MKSo0tYTRaUWPrcNq8tpA",
    client_secret="5twxWoEj6Y4fvik1SbqA7Sac105PNg",
    password="nitsawahied374",
    user_agent="price App",
    username="wahid028",
)

print(reddit.read_only)
# Output: False

False


In [23]:
print(reddit.user.me())

wahid028


### Main algorithm

In [74]:
import praw
from psaw import PushshiftAPI
import time
import os
import pandas as pd
import datetime as dt

# to use PSAW
api = PushshiftAPI()

# to use PRAW
reddit = praw.Reddit(
    client_id="9MKSo0tYTRaUWPrcNq8tpA",
    client_secret="5twxWoEj6Y4fvik1SbqA7Sac105PNg",
    password="nitsawahied374",
    user_agent="price App",
    username="wahid028",
)

def log_action(action):
    print(action)
    return

subreddits = ['bitcoin', 'btc']
start_year = 2020
end_year = 2021

# directory on which to store the data
basecorpus = './my-dataset/'

for year in range(start_year, end_year+1):
    action = '[Year]' + str(year)
    log_action(action)
    
    dirpath = basecorpus + str(year)
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)
        
    # timestamps that define window of posts
    ts_after = int(dt.datetime(year, 1, 1).timestamp())
    ts_before = int(dt.datetime(year+1, 1, 1).timestamp())
    
    
    for subreddit in subreddits:
        start_time = time.time()
        
        action = '\t[Subeddit]' + subreddit
        log_action(action)
        
        subreddit_dir_path = dirpath + '/' + subreddit
        if os.path.exists(subreddit_dir_path):
            continue
        else:
            os.makedirs(subreddit_dir_path)
            
        submissions_csv_path = str(year) + '-' + subreddit + '-submission.csv'
        
        submissions_dict = {
            "id" : [],
            "url" : [],
            "title" : [],
            "score" : [],
            "num_comments": [],
            "created_utc" : [],
            "selftext" : [],
        }
        
        # use PSAW only to get id of submissions in time interval
        gen = api.search_submissions(
            after=ts_after,
            before=ts_before,
            filter=['id'],
            subreddit=subreddit,
            limit=100
        )
        
        # use PRAW to get actual info and traverse comment tree
        for submission_psaw in gen:
            # use psaw here
            submission_id = submission_psaw.d_['id']
            # use praw from now on
            submission_praw = reddit.submission(id=submission_id)

            submissions_dict["id"].append(submission_praw.id)
            submissions_dict["url"].append(submission_praw.url)
            submissions_dict["title"].append(submission_praw.title)
            submissions_dict["score"].append(submission_praw.score)
            submissions_dict["num_comments"].append(submission_praw.num_comments)
            submissions_dict["created_utc"].append(submission_praw.created_utc)
            submissions_dict["selftext"].append(submission_praw.selftext)
            
            submission_comments_csv_path = str(year) + '-' + subreddit + '-submission_' + submission_id + '-comments.csv'
            submission_comments_dict = {
                "comment_id" : [],
                "comment_parent_id" : [],
                "comment_body" : [],
                "comment_link_id" : [],
            }
            
            # extend the comment tree all the way
            submission_praw.comments.replace_more(limit=None)
            # for each comment in flattened comment tree
            for comment in submission_praw.comments.list():
                submission_comments_dict["comment_id"].append(comment.id)
                submission_comments_dict["comment_parent_id"].append(comment.parent_id)
                submission_comments_dict["comment_body"].append(comment.body)
                submission_comments_dict["comment_link_id"].append(comment.link_id)

            # for each submission save separate csv comment file
            pd.DataFrame(submission_comments_dict).to_csv(subreddit_dir_path + '/' + submission_comments_csv_path,
                                                          index=False)
            
            # single csv file with all submissions
            pd.DataFrame(submissions_dict).to_csv(subreddit_dir_path + '/' + submissions_csv_path,
                                              index=False)


            action = f"\t\t[Info] Found submissions: {pd.DataFrame(submissions_dict).shape[0]}"
            log_action(action)

            action = f"\t\t[Info] Elapsed time: {time.time() - start_time: .2f}s"
            log_action(action)

[Year]2020
	[Subeddit]bitcoin


C:\Users\seokju gist\anaconda3\envs\mitoenv\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


		[Info] Found submissions: 1
		[Info] Elapsed time:  2.68s
		[Info] Found submissions: 2
		[Info] Elapsed time:  2.99s
		[Info] Found submissions: 3
		[Info] Elapsed time:  3.41s
		[Info] Found submissions: 4
		[Info] Elapsed time:  3.94s
		[Info] Found submissions: 5
		[Info] Elapsed time:  4.31s
		[Info] Found submissions: 6
		[Info] Elapsed time:  4.57s
		[Info] Found submissions: 7
		[Info] Elapsed time:  4.85s
		[Info] Found submissions: 8
		[Info] Elapsed time:  5.15s
		[Info] Found submissions: 9
		[Info] Elapsed time:  5.42s
		[Info] Found submissions: 10
		[Info] Elapsed time:  5.89s
		[Info] Found submissions: 11
		[Info] Elapsed time:  6.21s
		[Info] Found submissions: 12
		[Info] Elapsed time:  6.47s
		[Info] Found submissions: 13
		[Info] Elapsed time:  6.81s
		[Info] Found submissions: 14
		[Info] Elapsed time:  7.24s
		[Info] Found submissions: 15
		[Info] Elapsed time:  7.58s
		[Info] Found submissions: 16
		[Info] Elapsed time:  8.07s
		[Info] Found submissions: 17
		

		[Info] Found submissions: 34
		[Info] Elapsed time:  13.89s
		[Info] Found submissions: 35
		[Info] Elapsed time:  14.15s
		[Info] Found submissions: 36
		[Info] Elapsed time:  14.44s
		[Info] Found submissions: 37
		[Info] Elapsed time:  14.78s
		[Info] Found submissions: 38
		[Info] Elapsed time:  15.10s
		[Info] Found submissions: 39
		[Info] Elapsed time:  15.76s
		[Info] Found submissions: 40
		[Info] Elapsed time:  16.10s
		[Info] Found submissions: 41
		[Info] Elapsed time:  16.36s
		[Info] Found submissions: 42
		[Info] Elapsed time:  16.89s
		[Info] Found submissions: 43
		[Info] Elapsed time:  17.26s
		[Info] Found submissions: 44
		[Info] Elapsed time:  17.67s
		[Info] Found submissions: 45
		[Info] Elapsed time:  18.06s
		[Info] Found submissions: 46
		[Info] Elapsed time:  18.41s
		[Info] Found submissions: 47
		[Info] Elapsed time:  18.75s
		[Info] Found submissions: 48
		[Info] Elapsed time:  19.02s
		[Info] Found submissions: 49
		[Info] Elapsed time:  19.29s
		[Info]

		[Info] Found submissions: 66
		[Info] Elapsed time:  63.17s
		[Info] Found submissions: 67
		[Info] Elapsed time:  63.96s
		[Info] Found submissions: 68
		[Info] Elapsed time:  64.74s
		[Info] Found submissions: 69
		[Info] Elapsed time:  66.00s
		[Info] Found submissions: 70
		[Info] Elapsed time:  66.76s
		[Info] Found submissions: 71
		[Info] Elapsed time:  68.03s
		[Info] Found submissions: 72
		[Info] Elapsed time:  68.80s
		[Info] Found submissions: 73
		[Info] Elapsed time:  70.09s
		[Info] Found submissions: 74
		[Info] Elapsed time:  70.88s
		[Info] Found submissions: 75
		[Info] Elapsed time:  72.21s
		[Info] Found submissions: 76
		[Info] Elapsed time:  72.96s
		[Info] Found submissions: 77
		[Info] Elapsed time:  73.71s
		[Info] Found submissions: 78
		[Info] Elapsed time:  75.02s
		[Info] Found submissions: 79
		[Info] Elapsed time:  75.79s
		[Info] Found submissions: 80
		[Info] Elapsed time:  77.08s
		[Info] Found submissions: 81
		[Info] Elapsed time:  77.84s
		[Info]

		[Info] Found submissions: 99
		[Info] Elapsed time:  99.87s
		[Info] Found submissions: 100
		[Info] Elapsed time:  100.64s


In [1]:
# Documentation: https://praw.readthedocs.io/en/latest/ 
# https://www.youtube.com/watch?v=Y7BSe7EiBTs 
# https://towardsdatascience.com/how-to-collect-a-reddit-dataset-c369de539114

In [3]:
import pandas as pd
df = pd.read_csv(r'C:\Users\seokju gist\my-dataset\2021\btc\2021-btc-submission.csv')
df.head()

,id,url,title,score,num_comments,created_utc,selftext
0,rsveey,https://www.reddit.com/r/btc/comments/rsveey/will_us_regulators_shake_stablecoins_into/,Will US regulators shake stablecoins into high-tech banks?,1,0,1.640962e+09,[removed]
1,rsujie,https://www.reddit.com/r/btc/comments/rsujie/a_question_about_bch_volumes/,A question about BCH volumes.,30,88,1.640959e+09,"We know that BCH slipped from rank 5 to rank 25. I'm not here to cry about that, I'm just curious to understand how the 5th coin for volumes is the 25th coin for capitalization. Shouldn't the usage be proportional to his capitalization?\n\n(now it's 4th, but it was 5th few hours ago)\n\n&#x200B;\n\nhttps://preview.redd.it/zi76uesbxv881.png?width=1383&format=png&auto=webp&s=da9173ffdce2f30c58810c1064b48965dcf4e61b"
2,rstlbw,NaN,Looking for Free Bitcoin Mining Unlimited Platform,1,0,1.640956e+09,[removed]
3,rsthhk,NaN,Can We trust Tether? Everything You Need to Know About Tether (USDT),10,17,1.640956e+09,[deleted]
4,rstcy6,NaN,[deleted by user],1,4,1.640955e+09,[removed]
